In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import math

# Fast Text on OtherContext
- year_of_publication 
- Publisher
- number_of_pages 
- awards

In [ ]:
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"
df_books_temp = pd.read_csv(books, sep = "|")
df_books = df_books_temp.copy()
df_books.head(1)

In [ ]:
#We keep only few columns to create the context of training
df1 = df_books[df_books["book_language"] == "en"]
- year_of_publication 
- Publisher 
- number_of_pages 
- awards
df1 = df1[["Publisher", "awards"]]
print(len(df_books))
print(len(df1))
df1[1000:1005]

# Concatenation of embeddings

### Desc_Bert_768 || AuthorContext_FastText_60

##### Data merge




In [ ]:
df_bert_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")
df_fastText_auth_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/authorsContext_fastText.csv", sep = "|")

In [ ]:
df_bert = df_bert_temp.copy()
df_fastText_auth = df_fastText_auth_temp.copy()
print(len(df_bert))
print(len(df_fastText_auth))


105508
105508


In [ ]:
df_bert.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
#We only keep the embeddings
X1 = df_bert[["isbn", "book_language", "description_bert"]]
s1 = 768

X2 = df_fastText_auth["authorsContext_fastText"]
s2 = 60

df_embed = pd.concat([X1, X2], axis = 1)
print(len(df_embed))
df_embed.head()

105508


,isbn,book_language,description_bert,authorsContext_fastText
0,0439023483,en,0,"['-0.10757246970929005', '-0.09679322073085082..."
1,0316015849,en,"['-0.098250076', '-0.38007897', '0.8143984', '...","['-0.11777790052996527', '-0.08616064630088306..."
2,0525478817,en,"['-0.099239185', '-0.12767254', '0.6324886', '...","['-0.11273428555127353', '-0.08976304471012085..."
3,1416524797,en,"['-0.13360432', '-0.23354772', '0.47973692', '...","['-0.11270047592309614', '-0.08497616087940212..."
4,0062024035,en,"['-0.26884735', '-0.1411476', '0.30364788', '-...","['-0.13137996882200242', '-0.07718185376375913..."


In [ ]:
#ratings of Book Crossing
df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv", sep = ";", error_bad_lines=False, encoding="latin-1")
#df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv", sep = "|", error_bad_lines=False, encoding="latin-1")
df_ratings = df_ratings_temp.copy()

In [ ]:
#Merge between embeddings and ratings dataframe
df_embed = pd.merge(df_ratings, df_embed, left_on = "ISBN", right_on = "isbn")  
print(len(df_embed))
df_embed.head(1)

110801


,User-ID,ISBN,Book-Rating,isbn,book_language,description_bert,authorsContext_fastText
0,276725,034545104X,0,034545104X,en,"['-0.20497975', '0.032238785', '0.0620426', '-...","['-0.11242394912470546', '-0.08424112821618716..."


In [ ]:
#Verification of not empty lines
lineOK = df_embed[ (df_embed["book_language"] == "en") & \
                   (df_embed["description_bert"] != '0') & \
                   (df_embed["authorsContext_fastText"] != '0')]
print(len(lineOK))

63264


##### Strings of embeddings transformed into float

In [ ]:
#Transformation of a line of df_embed in a vector of float
#beacause bert embedding has been saved as a list of strings

MIN = min(df_embed["User-ID"])
MAX = max(df_embed["User-ID"])

def vectEmbed(l, s1, s2):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  b = np.zeros(s1 + s2 + 1)

  #1st copy of User-ID
  b[0] = (l['User-ID']-MIN)/(MAX-MIN)

  #Then handling of list of strings
  mi = -24.282314
  ma = 2.1074116
  v1 = l['description_bert'].split("'")
  b[1:(s1+1)] = [float(v1[i]) for i in range(1, len(v1), 2)]
  #b[1:(s1+1)] = [((float(v1[i]) - mi)/ (ma - mi)) for i in range(1, len(v1), 2)]

  mii = -0.6839812078202764
  maa = 0.6203103736042976
  v2 = l['authorsContext_fastText'].split("'")
  b[(s1+1):(s1 + s2 + 2)] = [float(v2[i]) for i in range(1, len(v2), 2)]
  #b[(s1+1):(s1 + s2 + 2)] = [((float(v2[i]) - mii)/ (maa - mii)) for i in range(1, len(v2), 2)]

  return b

In [ ]:
#print(df_embed["description_bert"][lineOK.index[0]])
#print("\n", df_embed["authorsContext_fastText"][lineOK.index[0]])
#print("\n", vectEmbed(df_embed.iloc[lineOK.index[0]], s1, s2))

In [ ]:
#treatment of all lines of df_embed 
#The result will be a X_data matrix

def matEmbed(l, s1, s2, df):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  m = np.zeros((len(l), (s1 + s2 + 1)))
  
  ind = 0
  for i in l.index:

      m[ind] = vectEmbed(df.iloc[i], s1, s2)
      ind = ind + 1
  
  return m

In [ ]:
#It takes about 1 min for 63 264 lines
mat_X_data = matEmbed(lineOK, s1, s2, df_embed)
mat_X_data.shape

(63264, 829)

##### Min / Max of embbedings values

In [ ]:
#First i performed the min on each lines corresponding to the book's description embedding
#=> so it remains a min for each column
print(mat_X_data[:, 1:(s1+1)].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, 1:(s1+1)].min(axis = 0))

(768,)


-24.282314

In [ ]:
#Same for the max
print(mat_X_data[:, 1:(s1+1)].max(axis = 0).shape)
max(mat_X_data[:, 1:(s1+1)].max(axis = 0))

(768,)


2.1074116

In [ ]:
#Same for the median
print(np.median(mat_X_data[:, 1:(s1+1)], axis = 0).shape)
np.median(np.median(mat_X_data[:, 1:(s1+1)], axis = 0))

(768,)


-0.0259018065

In [ ]:
#Same for the mean
print(np.mean(mat_X_data[:, 1:(s1+1)], axis = 0).shape)
np.mean(np.mean(mat_X_data[:, 1:(s1+1)], axis = 0))

(768,)


-0.039433989750688965

In [ ]:
nb_nb = mat_X_data[:, 1:(s1+1)].shape[0] * mat_X_data[:, 1:(s1+1)].shape[1]
r_m10 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] <= -10)).reshape(-1)) / (nb_nb), 2)
r_m10_m5 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > -10) & (mat_X_data[:, 1:(s1+1)] <= -5)).reshape(-1)) / (nb_nb), 2)
r_m5_m1 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > -5) & (mat_X_data[:, 1:(s1+1)] <= -1)).reshape(-1)) / (nb_nb), 2)
r_m1_0 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > -1) & (mat_X_data[:, 1:(s1+1)] <= 0)).reshape(-1)) / (nb_nb), 2)
r_0 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > 0)).reshape(-1)) / (nb_nb), 2)

print("Percentage of embeddings value under -10: \t\t", r_m10)
print("Percentage of embeddings value between -10 and -5: \t", r_m10_m5)
print("Percentage of embeddings value under -5 and -1: \t", r_m5_m1)
print("Percentage of embeddings value under -1 and 0: \t\t", r_m1_0)
print("Percentage of embeddings value above 0: \t\t", r_0)

Percentage of embeddings value under -10: 		 0.13
Percentage of embeddings value between -10 and -5: 	 0.0
Percentage of embeddings value under -5 and -1: 	 0.28
Percentage of embeddings value under -1 and 0: 		 52.1
Percentage of embeddings value above 0: 		 47.49


In [ ]:
#First i performed the min on each lines corresponding to the author context embedding
#=> so it remains a min for each column
print(mat_X_data[:, (s1+1):].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, (s1+1):].min(axis = 0))

(60,)


-0.6839812078202764

In [ ]:
#Same for the max
print(mat_X_data[:, (s1+1):].max(axis = 0).shape)
max(mat_X_data[:, (s1+1):].max(axis = 0))

(60,)


0.6203103736042976

### Desc_FastText_60 || AuthorContext_FastText_60

##### Data merge

In [ ]:
df_fastText_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_fastText.csv", sep = "|")
df_fastText_auth_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/authorsContext_fastText.csv", sep = "|")

In [ ]:
df_fastText = df_fastText_temp.copy()
df_fastText_auth = df_fastText_auth_temp.copy()
print(len(df_fastText))
print(len(df_fastText_auth))

105508
105508


In [ ]:
df_fastText.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_fastFM
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
#We only keep the embeddings
X1 = df_fastText[["isbn", "book_language", "description_fastFM"]]
s1 = 60

X2 = df_fastText_auth["authorsContext_fastText"]
s2 = 60

df_embed = pd.concat([X1, X2], axis = 1)
print(len(df_embed))
df_embed.head()

105508


,isbn,book_language,description_fastFM,authorsContext_fastText
0,0439023483,en,0,"['-0.10757246970929005', '-0.09679322073085082..."
1,0316015849,en,"['0.046032028402859235', '0.15718470524777384'...","['-0.11777790052996527', '-0.08616064630088306..."
2,0525478817,en,"['-0.015599380892056685', '0.34733651177241254...","['-0.11273428555127353', '-0.08976304471012085..."
3,1416524797,en,"['0.09067573049370284', '0.2065578283826736', ...","['-0.11270047592309614', '-0.08497616087940212..."
4,0062024035,en,"['-0.006060493551194668', '0.3125178421381861'...","['-0.13137996882200242', '-0.07718185376375913..."


In [ ]:
#ratings of Book Crossing
df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv", sep = ";", error_bad_lines=False, encoding="latin-1")
#df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv", sep = "|", error_bad_lines=False, encoding="latin-1")
df_ratings = df_ratings_temp.copy()

In [ ]:
#Merge between embeddings and ratings dataframe
df_embed = pd.merge(df_ratings, df_embed, left_on = "ISBN", right_on = "isbn")  
print(len(df_embed))
df_embed.head(1)

110801


,User-ID,ISBN,Book-Rating,isbn,book_language,description_fastFM,authorsContext_fastText
0,276725,034545104X,0,034545104X,en,"['-0.1532580568676903', '0.1413397986879794', ...","['-0.11242394912470546', '-0.08424112821618716..."


In [ ]:
#Verification of not empty lines
lineOK = df_embed[ (df_embed["book_language"] == "en") & \
                   (df_embed["description_fastFM"] != '0') & \
                   (df_embed["authorsContext_fastText"] != '0')]
print(len(lineOK))

63257


##### Strings of embeddings concatenated into float

In [ ]:
#Transformation of a line of df_embed in a vector of float
#beacause bert embedding has been saved as a list of strings

MIN = min(df_embed["User-ID"])
MAX = max(df_embed["User-ID"])

def vectEmbed2(l, s1, s2):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  b = np.zeros(s1 + s2 + 1)

  #1st copy of User-ID
  b[0] = (l['User-ID']-MIN)/(MAX-MIN)

  #Then handling of list of strings
  mi = -1.8926484078168868
  ma = 2.0181634426116943
  v1 = l['description_fastFM'].split("'")
  b[1:(s1+1)] = [float(v1[i]) for i in range(1, len(v1), 2)]
  #b[1:(s1+1)] = [((float(v1[i]) - mi) / (ma - mi)) for i in range(1, len(v1), 2)]

  mii = -0.6839812078202764
  maa = 0.6203103736042976
  v2 = l['authorsContext_fastText'].split("'")
  b[(s1+1):(s1 + s2 + 2)] = [float(v2[i]) for i in range(1, len(v2), 2)]
  #b[(s1+1):(s1 + s2 + 2)] = [((float(v2[i]) - mii)/ (maa - mii)) for i in range(1, len(v2), 2)]

  return b

In [ ]:
#treatment of all lines of df_embed 
#The result will be a X_data matrix

def matEmbed2(l, s1, s2, df):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  m = np.zeros((len(l), (s1 + s2 + 1)))
  
  ind = 0
  for i in l.index:

      m[ind] = vectEmbed2(df.iloc[i], s1, s2)
      ind = ind + 1
  
  return m

In [ ]:
#It takes about 1 min for 63 264 lines
mat_X_data = matEmbed2(lineOK, s1, s2, df_embed)
print(s1, s2, mat_X_data.shape)

60 60 (36009, 121)


##### Min / Max of embbedings values

In [ ]:
#First i performed the min on each lines corresponding to the book's description embedding
#=> so it remains a min for each column
print(mat_X_data[:, 1:(s1+1)].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, 1:(s1+1)].min(axis = 0))

(60,)


0.0

In [ ]:
#Same for the max
print(mat_X_data[:, 1:(s1+1)].max(axis = 0).shape)
max(mat_X_data[:, 1:(s1+1)].max(axis = 0))

(60,)


1.0

In [ ]:
#First i performed the min on each lines corresponding to the author context embedding
#=> so it remains a min for each column
print(mat_X_data[:, (s1+1):].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, (s1+1):].min(axis = 0))

(60,)


0.0

In [ ]:
#Same for the max
print(mat_X_data[:, (s1+1):].max(axis = 0).shape)
max(mat_X_data[:, (s1+1):].max(axis = 0))

(60,)


1.0

##### Strings of embeddings added as float

In [ ]:
#Transformation of a line of df_embed in a vector of float
#beacause bert embedding has been saved as a list of strings

MIN = min(df_embed["User-ID"])
MAX = max(df_embed["User-ID"])

def vectEmbed3(l, s):

  #for example fastText embedding of s = 60 added with fastText embedding of s = 60
  #          + User-ID
  b_temp = np.zeros(s + 1)
  b = np.zeros(s + 1)

  #1st copy of User-ID
  b_temp[0] = (l['User-ID']-MIN)/(MAX-MIN)

  #Then handling of list of strings
  mi = -1.8926484078168868
  ma = 2.0181634426116943
  v1 = l['description_fastFM'].split("'")
  b_temp[1:] = [((float(v1[i]) - mi) / (ma - mi)) for i in range(1, len(v1), 2)]

  mii = -0.6839812078202764
  maa = 0.6203103736042976
  v2 = l['authorsContext_fastText'].split("'")
  b[1:] = [((float(v2[i]) - mii)/ (maa - mii)) for i in range(1, len(v2), 2)]

  #Real addition (and no more concatenation) of both embeddings
  b[1:] = b[1:] + b_temp[1:]

  return b

In [ ]:
#treatment of all lines of df_embed 
#The result will be a X_data matrix

def matEmbed3(l, s, df):

  #for example fastText embedding of s = 60 added with fastText embedding of s = 60
  #          + User-ID
  m = np.zeros((len(l), (s + 1)))
  
  ind = 0
  for i in l.index:

      m[ind] = vectEmbed3(df.iloc[i], s)
      ind = ind + 1
  
  return m

In [ ]:
#It takes about 1 min for 63 264 lines
mat_X_data = matEmbed3(lineOK, 60, df_embed)
print(mat_X_data.shape)

(63257, 61)


##### Min / Max of embbedings values

In [ ]:
#First i performed the min on each lines corresponding to the book's description embedding
#=> so it remains a min for each column
print(mat_X_data[:, 1:].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, 1:].min(axis = 0))

(60,)


0.5091274849896965

In [ ]:
#Same for the max
print(mat_X_data[:, 1:].max(axis = 0).shape)
max(mat_X_data[:, 1:].max(axis = 0))

(60,)


1.5535841137550046

### Selection of corresponding labels

In [ ]:
Y_data = df_embed["Book-Rating"]
print(len(Y_data))
Y_data.head()

110801


0    0
1    5
2    0
3    5
4    9
Name: Book-Rating, dtype: int64

In [ ]:
#treatment of all corresponding lines of Y_data
#=> same lines as the one used to create mat_X_data

def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [ ]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(63264, 1)

# Train / Test split

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [ ]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 63264 

X_train.shape (50611, 829)
X_test.shape (12653, 829) 

y_train.shape (50611, 1)
y_test.shape (12653, 1)


# Ordinal regression

### Loading

In [ ]:
!pip install mord
from mord import LogisticAT, OrdinalRidge

  Created wheel for mord: filename=mord-0.6-cp37-none-any.whl size=6007 sha256=561d8cbe810a2703943dd356122c6b86243b7d5e5ef8a6e1d1782be621c77096
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


### Training / Test

In [ ]:
print(X_train[0:5].shape)
X_train[0:5]

(5, 61)


array([[0.        , 0.95768872, 1.05896814, 0.94864983, 0.89265968,
        0.9895396 , 1.04508977, 0.97021192, 1.19057702, 0.98438902,
        1.01870533, 1.03983308, 1.11396641, 0.98879508, 1.094478  ,
        1.09155859, 0.95588094, 1.00023446, 1.10045918, 1.06022601,
        1.00657103, 1.17663548, 1.09831444, 1.0365917 , 1.00450523,
        0.96964984, 1.06021591, 1.08512385, 1.03876455, 0.90716468,
        1.01060005, 1.04867544, 1.03888155, 1.06683467, 0.89026914,
        1.01118856, 1.05759605, 1.07389605, 1.00886036, 1.04795142,
        1.04338788, 1.21850834, 0.96355808, 0.96667674, 1.06476961,
        1.02015625, 0.98831335, 0.98438782, 0.91324193, 0.99379153,
        1.12872598, 0.93357488, 1.07627505, 1.00188875, 1.05329081,
        1.06298223, 1.06268194, 0.99568598, 0.84423213, 0.95383054,
        1.01899961],
       [0.        , 0.91022653, 0.99043207, 0.93361124, 0.91885099,
        0.92880451, 1.11580953, 1.04889048, 1.22632957, 0.99344997,
        0.98875665, 1.00796

In [ ]:
Counter(np.squeeze(y_train))

Counter({0: 14242, 1: 14565})

In [ ]:
#It takes about 35 min for desc_bert_768 || AuthorContext_FastText_60
LAT = LogisticAT(alpha=0., max_iter = 10000)
LAT.fit(X_train, np.squeeze(y_train))

ORD = OrdinalRidge(alpha=0., max_iter = 10000)
ORD.fit(X_train, np.squeeze(y_train))

OrdinalRidge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=10000,
             normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
X_test[0:5]

array([[ 2.79415878e-01,  4.43467140e-01,  6.01008635e-01,
        -7.24982907e-01, -9.42979306e-01,  4.48557703e-02,
        -2.42491813e-01,  2.56742516e-01,  8.45408845e-02,
         5.65251485e-02, -1.70130255e-01,  1.38299496e-02,
         3.01606947e-01,  1.18366673e-01,  1.42862273e-01,
        -4.30936627e-01,  7.25719258e-02,  9.15293515e-02,
         1.00632290e-01, -3.37962906e-01, -1.88919544e-01,
         3.93052990e-01, -3.98059500e-01, -4.10282910e-02,
         3.29827881e-01, -1.39744760e-01,  2.94060409e-02,
        -2.22906565e-01, -4.79535983e-01, -7.28794740e-01,
         1.73866279e-01,  6.12651334e-01, -1.76729295e-01,
         4.25761135e-01, -4.26812731e-01,  1.75532217e-01,
        -8.09199369e-02, -4.46537492e-01,  2.29345488e-01,
        -1.74934726e-01,  3.87857211e-01, -2.97395133e-01,
         1.58314399e-01,  1.66821264e-01,  1.82327340e-01,
         1.29737883e-01,  6.63058410e-02,  3.67454387e-01,
        -1.49679712e-01,  4.86236500e-01,  3.22266380e-0

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 3561, 1: 3641})

In [ ]:
y_pred_lat = LAT.predict(X_test)
y_pred_ord = ORD.predict(X_test)

### Unbalanced ratings

In [ ]:
print(Counter(y_pred_lat))

compRes_lat = [i for i in range(len(y_test)) if y_pred_lat[i] != y_test[i]]
print("Percentage of misclassified (LogisticAT): ", (len(compRes_lat) / len(y_test))*100)

Counter({0: 12194, 5: 186, 4: 86, 3: 74, 6: 43, 2: 36, 7: 19, 1: 14})
Percentage of misclassified (LogisticAT):  39.30603857097692


In [ ]:
print(Counter(y_pred_ord))

compRes_ord = [i for i in range(len(y_test)) if y_pred_ord[i] != y_test[i]]
print("Percentage of misclassified (OrdinalRidge): ", (len(compRes_ord) / len(y_test))*100)

Counter({3.0: 7914, 2.0: 2842, 4.0: 1726, 1.0: 118, 5.0: 52})
Percentage of misclassified (OrdinalRidge):  99.4783433449257


### Binary classes

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix_lat = confusion_matrix( y_pred_lat, y_test )
print("Confusion matrix (LogisticAT) :", 100*(confusion_matrix_lat/len(y_test)), sep = "\n")

print("Percentage of misclassified (LogisticAT): ", ((confusion_matrix_lat[0,1] + confusion_matrix_lat[1,0]) / len(y_test))*100)

Confusion matrix (LogisticAT) :
[[26.53429603 19.64732019]
 [22.91030269 30.90808109]]
Percentage of misclassified (LogisticAT):  42.557622882532634


In [ ]:
confusion_matrix_ord = confusion_matrix( y_pred_ord, y_test )
print("Confusion matrix (OrdinalRidge) :", 100*(confusion_matrix_ord/len(y_test)), sep = "\n")

print("Percentage of misclassified (OrdinalRidge): ", ((confusion_matrix_ord[0,1] + confusion_matrix_ord[1,0]) / len(y_test))*100)

Confusion matrix (OrdinalRidge) :
[[26.49264093 19.57789503]
 [22.95195779 30.97750625]]
Percentage of misclassified (OrdinalRidge):  42.52985281866148


# Classical regression

In [ ]:
from sklearn.linear_model import LinearRegression

### Unbalanced ratings

In [ ]:
def MyRound(x): 
  return math.ceil(x) if math.ceil(x) - x < x - math.floor(x) else math.floor(x)

In [ ]:
model_LinReg = LinearRegression()
model_LinReg.fit(X_train, np.squeeze(y_train))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_lin_pred = model_LinReg.predict(X_test)

In [ ]:
y_lin_pred = list(map(MyRound, y_lin_pred))
print(Counter(y_lin_pred))

compRes_lin = [i for i in range(len(y_test)) if y_lin_pred[i] != y_test[i]]
print("Percentage of misclassified (LinearRegression): ", (len(compRes_lin) / len(y_test))*100)

Counter({3: 5247, 2: 3793, 4: 2566, 1: 528, 5: 436, 6: 58, 7: 19, 0: 6})
Percentage of misclassified (LinearRegression):  99.3598356121078


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Training / Test

In [ ]:
#It takes about 3 min for desc_bert_768 || AuthorContext_FastText_60
modele_rf = RandomForestClassifier()
modele_rf.fit(X_train, np.squeeze(y_train))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_predict_rf = modele_rf.predict(X_test)

### Unbalanced ratings

In [ ]:
print(Counter(y_predict_rf))

compRes_rf = [i for i in range(len(y_test)) if y_predict_rf[i] != y_test[i]]
print("Percentage of misclassified (RandomForestClassifier): ", (len(compRes_rf) / len(y_test))*100)

Counter({0: 8473, 8: 1032, 10: 824, 7: 727, 9: 610, 5: 457, 6: 336, 4: 108, 3: 52, 2: 17, 1: 17})
Percentage of misclassified (RandomForestClassifier):  54.99881451039279


### Binary classes

In [ ]:
confusion_matrix_rf = confusion_matrix( y_predict_rf, y_test )
print("Confusion matrix (RandomForestClassifier) :", 100*(confusion_matrix_rf/len(y_test)), sep = "\n")

print("Percentage of misclassified (RandomForestClassifier): ", ((confusion_matrix_rf[0,1] + confusion_matrix_rf[1,0]) / len(y_test))*100)

Confusion matrix (RandomForestClassifier) :
[[25.77061927 21.8689253 ]
 [23.67397945 28.68647598]]
Percentage of misclassified (RandomForestClassifier):  45.542904748680925


# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_LogReg = LogisticRegression(max_iter = 10000, solver = "liblinear")

In [ ]:
model_LogReg.fit(X_train, np.squeeze(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_predict_LogReg = model_LogReg.predict(X_test)

In [ ]:
confusion_matrix_LogReg = confusion_matrix( y_predict_LogReg, y_test )
print("Confusion matrix (LogisticRegression) :", 100*(confusion_matrix_LogReg/len(y_test)), sep = "\n")

print("Percentage of misclassified (LogisticRegression): ", ((confusion_matrix_LogReg[0,1] + confusion_matrix_LogReg[1,0]) / len(y_test))*100)

Confusion matrix (LogisticRegression) :
[[26.14551513 19.49458484]
 [23.29908359 31.06081644]]
Percentage of misclassified (LogisticRegression):  42.79366842543738
